In [ ]:
import pandas as pd
import numpy as np 
import os as os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
from sklearn.cluster import KMeans
from scipy.stats import ttest_ind 
from scipy.stats import fisher_exact

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [ ]:
ecmodf = pd.read_csv('data/ecmo.csv', index_col=0)
ecmodf = ecmodf.drop(columns=['sx_v', 'neut', 'cr'])
print(ecmodf.isnull().sum())

## Iterative round robin regression imputation maintains original clusters

In [ ]:
imp = IterativeImputer(verbose=2, max_iter=20, min_value=0)
ecmodf_impute = ecmodf.copy()
imp.fit(ecmodf_impute)

In [ ]:
ecmodf = pd.DataFrame(imp.transform(ecmodf_impute), columns = ecmodf.columns, index = ecmodf.index)

In [ ]:
print(ecmodf.isnull().sum())

In [ ]:
ecmodf['bmi'] = ecmodf['bmi'].astype('int64')
ecmodf.head()

In [ ]:
ecmodf_norm = ecmodf.copy()

ecmodf_norm['ddim'] = np.log(ecmodf_norm['ddim']) #log conversion of skewed variables - days and sofa not included as would not expected to conform to a power law distribution
ecmodf_norm['ferritin'] = np.log(ecmodf_norm['ferritin'])
ecmodf_norm['pct'] = np.log(ecmodf_norm['pct'])
ecmodf_norm['nlrat'] = np.log(ecmodf_norm['nlrat'])
ecmodf_norm['lymph'] = np.log(ecmodf_norm['lymph'])
ecmodf_norm['pplat'] = np.log(ecmodf_norm['pplat'])
ecmodf_norm['pco2'] = np.log(ecmodf_norm['pco2'])
ecmodf_norm['bmi'] = np.log(ecmodf_norm['bmi'])
ecmodf_norm['sofa'] = np.log(ecmodf_norm['sofa'])

ecmodf_norm.head()

In [ ]:
normalize = preprocessing.MinMaxScaler(feature_range=(0.01, 1.01))
ecmodf_normalized = normalize.fit_transform(ecmodf_norm)
ecmodf_normalized = pd.DataFrame(ecmodf_normalized)
ecmodf_normalized.columns = ecmodf_norm.columns
ecmodf_normalized.head()

## general metrics

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# A list holds the silhouette coefficients for each k
silhouette_coefficients = []

# Only works for 2 and above clusters as assesses inter-centroid distance
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, init='k-means++')
    kmeans.fit(ecmodf_normalized)
    score = silhouette_score(ecmodf_normalized, kmeans.labels_)
    silhouette_coefficients.append(score)
    
plt.style.use("fivethirtyeight")
plt.plot(range(2, 10), silhouette_coefficients)
plt.xticks(range(2, 10))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
from validclust.validclust import ValidClust #rapid assessment of optimal k using multiple tests

data_temp = ecmodf_normalized.to_numpy()
vclust = ValidClust(k=list(range(2, 9)), methods=['kmeans'])

cvi_vals = vclust.fit_predict(data_temp)
print(cvi_vals)

In [ ]:
vclust.plot() # darker cells indicate higher quality clustering

## cluster prediction strength 

In [ ]:
## https://towardsdatascience.com/prediction-strength-a-simple-yet-relatively-unknown-way-to-evaluate-clustering-2e5eaf56643
## Tibshirani R, Walther G. Cluster validation by prediction strength. J Comput Graph Stat 2005; 3: 511e28
## Used in 10.1016/j.bja.2019.02.026

In [ ]:
import sys

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

from scipy.spatial import distance

import matplotlib.pyplot as plt
import seaborn as sns 
plt.style.use('seaborn')

In [ ]:
# setting the range of k
clusters = range(1, 10)

# running the clustering 
wss_list = []

for k in clusters:
    model = KMeans(n_clusters=k)
    model.fit(ecmodf_normalized)
    wss_list.append(model.inertia_)

# plotting
_, ax = plt.subplots()
ax.plot(clusters, wss_list, '-o', color='black')
ax.set(title='Elbow plot', 
       xlabel='number of clusters', 
       ylabel='WSS');

In [ ]:
temp2 = ecmodf_normalized.copy()
temp2.head()

In [ ]:
# train/test split
data_temp2 = ecmodf_normalized.to_numpy()
X_train, X_test = train_test_split(data_temp2, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
## helper function to define closest centroid to given observation

def get_closest_centroid(obs, centroids):
    '''
    Function for retrieving the closest centroid to the given observation 
    in terms of the Euclidean distance.
    
    Parameters
    ----------
    obs : array
        An array containing the observation to be matched to the nearest centroid
    centroids : array
        An array containing the centroids
    
    Returns
    -------
    min_centroid : array
        The centroid closes to the obs 
    '''
    min_distance = sys.float_info.max
    min_centroid = 0
    
    for c in centroids:
        dist = distance.euclidean(obs, c)
        if dist < min_distance:
            min_distance = dist
            min_centroid = c
            
    return min_centroid

In [ ]:
## main function that determines prediction strength of each cluster

def get_prediction_strength(k, train_centroids, x_test, test_labels):
    '''
    Function for calculating the prediction strength of clustering
    
    Parameters
    ----------
    k : int
        The number of clusters
    train_centroids : array
        Centroids from the clustering on the training set
    x_test : array
        Test set observations
    test_labels : array
        Labels predicted for the test set
        
    Returns
    -------
    prediction_strength : float
        Calculated prediction strength
    '''
    n_test = len(x_test)
    
    # populate the co-membership matrix
    D = np.zeros(shape=(n_test, n_test))
    for x1, l1, c1 in zip(x_test, test_labels, list(range(n_test))):
        for x2, l2, c2 in zip(x_test, test_labels, list(range(n_test))):
            if tuple(x1) != tuple(x2):
                if tuple(get_closest_centroid(x1, train_centroids)) == tuple(get_closest_centroid(x2, train_centroids)):
                    D[c1,c2] = 1.0
    
    # calculate the prediction strengths for each cluster
    ss = []
    for j in range(k):
        s = 0
        examples_j = x_test[test_labels == j, :].tolist()
        n_examples_j = len(examples_j)
        for x1, l1, c1 in zip(x_test, test_labels, list(range(n_test))):
            for x2, l2, c2 in zip(x_test, test_labels, list(range(n_test))):
                if tuple(x1) != tuple(x2) and l1 == l2 and l1 == j:
                    s += D[c1,c2]
        ss.append(s / (n_examples_j * (n_examples_j - 0.9999))) 

    prediction_strength = min(ss)

    return prediction_strength

In [ ]:
# running the clustering 
strengths = []
for k in clusters:
    model_train = KMeans(n_clusters=k, random_state=42).fit(X_train)
    model_test = KMeans(n_clusters=k, random_state=42).fit(X_test)
    
    pred_str = get_prediction_strength(k, model_train.cluster_centers_, X_test, model_test.labels_)
    strengths.append(pred_str)

# plotting
_, ax = plt.subplots()
ax.plot(clusters, strengths, '-o', color='black')
ax.axhline(y=0.8, c='red');
ax.set(title='Determining the optimal number of clusters', 
       xlabel='number of clusters', 
       ylabel='prediction strength');

##Should select maximum cluster for which prediction strength is above certain theshold (i.e. 3 in this case)

## elbow and gap statistic

In [ ]:
inertia = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k, init='k-means++')
    kmeanModel.fit(ecmodf_normalized)
    inertia.append(kmeanModel.inertia_)
    
plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
from kneed import KneeLocator

kl = KneeLocator(
    range(1, 10), inertia, curve="convex", direction="decreasing"
)

kl.elbow

In [ ]:
from gap_statistic import OptimalK

optimalK = OptimalK(parallel_backend=None) #multicore
optimalK

In [ ]:
n_clusters = optimalK(ecmodf_normalized, cluster_array=np.arange(1, 10))
print('Optimal clusters: ', n_clusters)

In [ ]:
optimalK.gap_df

In [ ]:
plt.plot(optimalK.gap_df.n_clusters, optimalK.gap_df.gap_value, linewidth=3)
plt.scatter(optimalK.gap_df[optimalK.gap_df.n_clusters == n_clusters].n_clusters,
            optimalK.gap_df[optimalK.gap_df.n_clusters == n_clusters].gap_value, s=250, c='r')
plt.grid(True)
plt.xlabel('Cluster Count')
plt.ylabel('Gap Value')
plt.title('Gap Values by Cluster Count')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, init='k-means++', n_init=1000, n_jobs=-1).fit(ecmodf_normalized)
centroids = kmeans.cluster_centers_
print(centroids)

plt.scatter(ecmodf_normalized['ferritin'], ecmodf_normalized['ddim'], c= kmeans.labels_.astype(float), s=50, alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)
plt.show()

In [ ]:
pred = kmeans.predict(ecmodf_normalized)
ecmodf_clustered = pd.DataFrame(ecmodf)
ecmodf_clustered['cluster'] = pred

In [ ]:
ecmodf_clustered['cluster'].value_counts()

In [ ]:
ecmodf_clustered